In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [11]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
houseprice_df = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

houseprice_df.head(10)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,None,IR1,Lvl,AllPub,...,0,None,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,None,IR1,Lvl,AllPub,...,0,None,None,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,1,2008,WD,Normal,118000


In [12]:
import statsmodels.api as sm

Y = houseprice_df['saleprice']
X = houseprice_df[['overallqual','grlivarea','garagecars','garagearea','totalbsmtsf','firstflrsf']]

X = sm.add_constant(X)

result = sm.OLS(Y,X).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     775.0
Date:                Mon, 19 Aug 2019   Prob (F-statistic):               0.00
Time:                        12:34:14   Log-Likelihood:                -17496.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1453   BIC:                         3.504e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.027e+05   4903.994    -20.932      

In [13]:
#assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.
#Do you think your model is satisfactory? If so, why?

In [14]:
#The F-test 775.0 and its p-value close to zero are pretty satisfactory.
#also, r-squared, adjusted r-squared values are not that bad either, 0.76.
#However, the model can definitely improve by including interaction terms and/or missing explanatory variables.
#I cannot comment on AIC, BIC values above as I do not have other model to compare those values to.

In [15]:
#In order to improve the goodness of fit of your model, try different model specifications by adding or removing 
#some variables.
#For each model you try, get the goodness of fit metrics and compare your models with each other. 
#Which model is the best and why?

In [16]:
houseprice_df['overall_garage']=houseprice_df['overallqual']*houseprice_df['garagecars']

In [20]:
houseprice_df['overall_size']=houseprice_df['totalbsmtsf']*houseprice_df['firstflrsf']

In [22]:
Y = houseprice_df['saleprice']
X = houseprice_df[['grlivarea','garagearea','overall_size','overall_garage']]

X = sm.add_constant(X)

result = sm.OLS(Y,X).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.756
Method:                 Least Squares   F-statistic:                     1130.
Date:                Mon, 19 Aug 2019   Prob (F-statistic):               0.00
Time:                        12:36:05   Log-Likelihood:                -17513.
No. Observations:                1460   AIC:                         3.504e+04
Df Residuals:                    1455   BIC:                         3.506e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           2.808e+04   3523.757      7.

In [24]:
#in this model, I have included interaction term for garagecars and overallqual variables.
#I have also included the interaction term for firstflrsf and totalbsmsf variables.
#this made garagearea to become more statistically significant.
#r-squared and adjusted r-squared values went down. I'm not sure if it's because the number of 
#explanatory variables decreased.
#AIC, BIC values are about the same, so I cannot determine which model is better.
#I will try adding more of the missing variable that might affect the houseprice.

In [30]:
houseprice_df['salecondition'].unique()

array(['Normal', 'Abnorml', 'Partial', 'AdjLand', 'Alloca', 'Family'],
      dtype=object)

In [31]:
Y = houseprice_df['saleprice']
X = houseprice_df[['grlivarea','garagearea','overall_size','overall_garage', 'yrsold']]

X = sm.add_constant(X)

result = sm.OLS(Y,X).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.756
Method:                 Least Squares   F-statistic:                     903.3
Date:                Mon, 19 Aug 2019   Prob (F-statistic):               0.00
Time:                        12:47:00   Log-Likelihood:                -17513.
No. Observations:                1460   AIC:                         3.504e+04
Df Residuals:                    1454   BIC:                         3.507e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -5.601e+05   1.56e+06     -0.

In [32]:
#poolarea, mosold, yrsold all had statistically low t-test results.  R-squared, adjusted r-squared values
#didn't improve either.  AIC, BIC did not improve either.

In [34]:
#Overall, the first model seems to be better than the other two when comparing r-squared, adjusted r-squared, AIC,
#BIC.  